# AI Techno Schubert

Create Techno Schubert tracks after training on 8 of Schubert's compositions encoded as MIDI files.

Start by getting the tracks and a working copy of the model

Sources:
https://medium.com/@leesurkis/how-to-generate-techno-music-using-deep-learning-17c06910e1b3

https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5

In [1]:
# clean up
!rm -Rf AI_schubert ;
# clone the repo
!git clone https://github.com/douglasgoodwin/AI_schubert ;

# clean up the MIDI files
!rm -Rf schubert ;
!rm -Rf bach ;
!rm -Rf techno ;
!rm -Rf data ;
!rm -Rf weights ;

!mv AI_schubert/schubert ./ ;
!mv AI_schubert/bach ./ ;
!mv AI_schubert/techno ./ ;

# move all the models
!mv AI_schubert/*.h5 ./ ;

# move the weights and data
!mv AI_schubert/weights ./ ;
!mv AI_schubert/data ./ ;

# cleanups
!mv *.hdf5 weights/ ;

Cloning into 'AI_schubert'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 97 (delta 2), reused 11 (delta 2), pack-reused 86
Unpacking objects: 100% (97/97), 90.37 MiB | 6.05 MiB/s, done.
mv: cannot stat '*.hdf5': No such file or directory


In [2]:
# Set up some variables
# mymodel = 'best_model.h5'
# mymodel = 'bach_model.h5'
midisongs = "techno/*.mid"
TRAIN = True

In [3]:
# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install some software

In [4]:
!pip install music21

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
#library for understanding music
from music21 import converter, instrument, note, chord, stream

import glob
import pickle
import numpy
from os.path import exists
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# Train a model

In [ ]:
# This code is cleaner that the original
# https://github.com/leesurkis/generating-techno-music/blob/master/Generating_Techno_Music-Train.ipynb


def train_network():
    """This function calls all other functions and trains the LSTM"""
    print("===== train_network() ======")

    notes = get_notes()

    # get amount of pitch names
    n_vocab = len(set(notes))
    network_input, network_output = prepare_sequences(notes, n_vocab)
    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)


def get_notes():
    """Extracts all notes and chords from midi files in the midisongs
    directory and creates a file with all notes in string format"""

    print("===== get_notes() ======")
    notes = []

    for file in glob.glob(midisongs):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append(".".join(str(n) for n in element.normalOrder))

    with open("data/notes", "wb") as filepath:
        pickle.dump(notes, filepath)

    return notes


def prepare_sequences(notes, n_vocab):
    """Prepare the sequences which are the inputs for the LSTM"""

    print("===== prepare_sequences() ======")
    # sequence length should be changed after experimenting with different numbers
    sequence_length = 30

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)


def create_network(network_input, n_vocab):
    """Creates the structure of the neural network.
    Refactored to work for both training and prediction drg"""

    print("===== create_network() ======")
    model = Sequential()
    model.add(
        LSTM(
            512,
            input_shape=(network_input.shape[1], network_input.shape[2]),
            return_sequences=True,
        )
    )
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

    # Load the weights to each node (if weights file is available)
    # Now This function should also serve predictions drg
    weightsfile = 'weights/weights-improvement-30-0.4634-bigger.hdf5'
    # weightsfile = "/content/weights-improvement-14-3.3951-bigger.hdf5"
    if exists(weightsfile):
      model.load_weights(weightsfile)

    return model

    


def train(model, network_input, network_output):
    """train the neural network"""

    print("===== train() ======")
    filepath = "weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath, monitor="loss", verbose=0, save_best_only=True, mode="min"
    )
    callbacks_list = [checkpoint]

    # experiment with different epoch sizes and batch sizes
    # 15 epochs may be enough
    model.fit(
        network_input,
        network_output,
        epochs=15,
        batch_size=64,
        callbacks=callbacks_list,
    )

if TRAIN:
  print("===== TRAINING! ======")
  train_network()

/usr/local/lib/python3.9/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=7, channel=None, data=b'sequenc\xe9 par '>; getting generic Instrument
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=8, channel=None, data=b'Jean-Nicholas Ren\xe9  97'>; getting generic Instrument
  warnings.warn(


Parsing techno/d-desire.mid
Parsing techno/blkisblk_d.mid
Parsing techno/d-babybaby.mid
Parsing techno/d-bemylover.mid
Parsing techno/d-anything.mid
Parsing techno/d-becauseimlovingyou.mid
Parsing techno/d-dontgoaway.mid
Parsing techno/d-badboy.mid
Parsing techno/d-allthatshewants.mid
Parsing techno/d-children.mid
Parsing techno/d-cometakemyhand.mid
Parsing techno/d-awayfromhome.mid
Parsing techno/d-celebratethelove.mid
Parsing techno/d-anotherday.mid
Parsing techno/d-closetoyou.mid
Parsing techno/d-123trainwithme.mid
Parsing techno/d-dontcryformeargentina.mid
Parsing techno/d-dadip.mid
Parsing techno/d-beautifullife.mid
Parsing techno/d-childrenofthenight.mid
Parsing techno/d-captainjack.mid
Parsing techno/d-100purelove.mid
Parsing techno/d-dafunk.mid
Parsing techno/blowthe_d.mid
Parsing techno/d-comenrideit.mid
Parsing techno/d-anothernight.mid
Parsing techno/borntobewild2.mid
Parsing techno/celebrat_d.mid
Parsing techno/d-dancingwithanangel.mid
Parsing techno/d-automaticlover.mid


# Start here if you don't need to train the model.

In [ ]:
def generate():
    """Generates the midi file"""
    print("===== generate the MIDI file ======")
    # load the notes used to train the model
    with open("data/notes", "rb") as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)


def prepare_sequences(notes, pitchnames, n_vocab):
    """Prepare the sequences used by the Neural Network"""

    print("===== prepare_sequences ======")
    # map back from integers to notes
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i : i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)


# def create_network(network_input, n_vocab):
#     """ Creates the structure of the neural network """

#     model = Sequential()
#     model.add(LSTM(
#         512,
#         input_shape=(network_input.shape[1], network_input.shape[2]),
#         return_sequences=True
#     ))
#     model.add(Dropout(0.3))
#     model.add(LSTM(512, return_sequences=True))
#     model.add(Dropout(0.3))
#     model.add(LSTM(512))
#     model.add(Dense(256))
#     model.add(Dropout(0.3))
#     model.add(Dense(n_vocab))
#     model.add(Activation('softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#     # Load the weights to each node
#     weightsfile = 'weights/weights-improvement-30-0.4634-bigger.hdf5'
#     if exists(weightsfile):
#       model.load_weights('weights/weights-improvement-30-0.4634-bigger.hdf5')

#     return model


def generate_notes(model, network_input, pitchnames, n_vocab):
    """Generate notes from the neural network based on a sequence of notes"""

    print("===== generate_notes ======")
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input) - 1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 new notes. See diagram:
    # https://miro.medium.com/v2/resize:fit:736/format:webp/1*ACKdHn6zR86lynlr_VEcGw.png

    # The code below shows how the model generates 500 notes.
    # Create a numpy array of all of the predictions,
    # and then index the note with the highest probability.
    # This note is then the prediction,
    # Append to the input sequence in the next iteration

    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)  # numpy array of predictions
        result = int_to_note[index]  # indexing the note with the highest probability
        prediction_output.append(result)  # that note is the prediction output

        pattern.append(index)
        pattern = pattern[1 : len(pattern)]

    return prediction_output


def create_midi(prediction_output):

    """Converts the output from the prediction to notes and create a midi file
    from the notes"""

    print("===== write the MIDI file ======")
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ("." in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split(".")
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.SnareDrum()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.SnareDrum()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write("midi", fp="test_output2.mid")

In [ ]:
# Generate!
generate()

---

# Ignore this below -- still refactoring

In [ ]:
# # Call the pickled notes
# with open("data/notes", "rb") as filepath:
#     notes = pickle.load(filepath)

# # prepare the sequences as in the data preparation section
# sequence_length = 100
# network_input = []
# output = []
# for i in range(0, len(notes) - sequence_length, 1):
#     sequence_in = notes[i : i + sequence_length]
#     sequence_out = notes[i + sequence_length]
#     network_input.append([note_to_int[char] for char in sequence_in])
#     output.append(note_to_int[sequence_out])



# prediction_output = []
# for note_index in range(500):
#     prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
#     prediction_input = prediction_input / float(n_vocab)

# prediction = model.predict(prediction_input, verbose=0)
# index = numpy.argmax(prediction)

# result = int_to_note[index]
# prediction_output.append(result)
# pattern.append(index)
# pattern = pattern[1 : len(pattern)]

# # convert the output back into notes and create a midi file.
# for pattern in prediction_output:
#     if ("." in pattern) or pattern.isdigit():
#         notes_in_chord = pattern.split(".")
#         notes = []
#         for current_note in notes_in_chord:
#             new_note = note.Note(int(current_note))
#             new_note.storedInstrument = instrument.SnareDrum()
#             notes.append(new_note)
#         new_chord = chord.Chord(notes)
#         new_chord.offset = offset
#         output_notes.append(new_chord)
#     else:
#         new_note = note.Note(pattern)
#         new_note.offset = offset
#         new_note.storedInstrument = instrument.SnareDrum()
#         output_notes.append(new_note)

# offset += 0.5

In [ ]:
# #defining function to read MIDI files
# def read_midi(file):
    
#     print("Loading Music File:",file)
    
#     notes=[]
#     notes_to_parse = None
    
#     #parsing a midi file
#     midi = converter.parse(file)
  
#     #grouping based on different instruments
#     s2 = instrument.partitionByInstrument(midi)

#     #Looping over all the instruments
#     for part in s2.parts:
#       # let's extract all parts
#       notes_to_parse = part.recurse() 

#       #finding whether a particular element is note or a chord
#       for element in notes_to_parse:
          
#           #note
#           if isinstance(element, note.Note):
#               notes.append(str(element.pitch))
          
#           #chord
#           elif isinstance(element, chord.Chord):
#               notes.append('.'.join(str(n) for n in element.normalOrder))

#     return np.array(notes)

# Load the MIDI files

In [ ]:
# #for listing down the file names
# import os

# #Array Processing
# import numpy as np

# #specify the path
# # path='schubert/'
# path='/content/bach/'

# #read all the filenames
# files=[i for i in os.listdir(path) if i.endswith(".mid")]

# #reading each midi file
# notes_array = np.array([read_midi(path+i) for i in files])

In [ ]:
# def get_notes_chords_rests(path):
#     try:
#         midi = converter.parse(path)
#         parts = instrument.partitionByInstrument(midi)
#         note_list = []
#         for music_instrument in range(len(parts)):
#           for element_by_offset in stream.iterator.OffsetIterator(parts[music_instrument]):
#               for entry in element_by_offset:
#                   if isinstance(entry, note.Note):
#                       note_list.append(str(entry.pitch))
#                   elif isinstance(entry, chord.Chord):
#                       note_list.append('.'.join(str(n) for n in entry.normalOrder))
#                   elif isinstance(entry, note.Rest):
#                       note_list.append('Rest')
#         return note_list
#     except Exception as e:
#         print("failed on ", path)
#         pass

# get_notes_chords_rests("/content/bach/invent8.mid")

In [ ]:
print(notes_array)

In [ ]:
#converting 2D array into 1D array
notes_ = [element for note_ in notes_array for element in note_]

#No. of unique notes
unique_notes = list(set(notes_))
print(len(unique_notes))

# Quick analysis of the notes

In [ ]:
#importing library
from collections import Counter

#computing frequency of each note
freq = dict(Counter(notes_))

#library for visualiation
import matplotlib.pyplot as plt

#consider only the frequencies
no=[count for _,count in freq.items()]

#set the figure size
plt.figure(figsize=(5,5))

#plot
plt.hist(no)

In [ ]:
frequent_notes = [note_ for note_, count in freq.items() if count>=50]
print(len(frequent_notes))

In [ ]:
# new_music=[]

# for notes in notes_array:
#     temp=[]
#     for note_ in notes:
#         if note_ in frequent_notes:
#             temp.append(note_)            
#     new_music.append(temp)
    
# new_music = np.array(new_music)

In [ ]:
# no_of_timesteps = 32
# x = []
# y = []

# for note_ in new_music:
#     for i in range(0, len(note_) - no_of_timesteps, 1):
        
#         #preparing input and output sequences
#         input_ = note_[i:i + no_of_timesteps]
#         output = note_[i + no_of_timesteps]
        
#         x.append(input_)
#         y.append(output)
        
# x=np.array(x)
# y=np.array(y)

In [ ]:
# unique_x = list(set(x.ravel()))
# x_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_x))

In [ ]:
# #preparing input sequences
# x_seq=[]
# for i in x:
#     temp=[]
#     for j in i:
#         #assigning unique integer to every note
#         temp.append(x_note_to_int[j])
#     x_seq.append(temp)
    
# x_seq = np.array(x_seq)

In [ ]:
# def lstm():
#   # model = Sequential()
#   # model.add(LSTM(128,return_sequences=True))
#   # model.add(LSTM(128))
#   # model.add(Dense(256))
#   # model.add(Activation('relu'))
#   # model.add(Dense(n_vocab))
#   # model.add(Activation('softmax'))
#   # model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

#   # try this:
#   # https://medium.com/@leesurkis/how-to-generate-techno-music-using-deep-learning-17c06910e1b3
#   model = Sequential()
#   model.add(
#     LSTM(
#       512,
#       input_shape=(network_input.shape[1], network_input.shape[2]),
#       return_sequences=True
#     )
#   )
#   model.add(Dropout(0.3))
#   model.add(LSTM(512, return_sequences=True))
#   model.add(Dropout(0.3))
#   model.add(LSTM(512))
#   model.add(Dense(256))
#   model.add(Dropout(0.3))
#   model.add(Dense(n_vocab))
#   model.add(Activation('softmax'))
#   model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#   return model

In [ ]:
# # Similarly, prepare the integer sequences for output data as well

# unique_y = list(set(y))
# y_note_to_int = dict((note_, number) for number, note_ in enumerate(unique_y)) 
# y_seq=np.array([y_note_to_int[i] for i in y])

# # Let us preserve 80% of the data for training and the rest 20% for the evaluation:
# from sklearn.model_selection import train_test_split
# x_tr, x_val, y_tr, y_val = train_test_split(x_seq,y_seq,test_size=0.2,random_state=0)


# Make the model

In [ ]:
# from keras.layers import *
# from keras.models import *
# from keras.callbacks import *
# import keras.backend as K

# K.clear_session()
# model = Sequential()
    
# #embedding layer
# model.add(Embedding(len(unique_x), 100, input_length=32,trainable=True)) 

# model.add(Conv1D(64,3, padding='causal',activation='relu'))
# model.add(Dropout(0.2))
# model.add(MaxPool1D(2))
    
# model.add(Conv1D(128,3,activation='relu',dilation_rate=2,padding='causal'))
# model.add(Dropout(0.2))
# model.add(MaxPool1D(2))

# model.add(Conv1D(256,3,activation='relu',dilation_rate=4,padding='causal'))
# model.add(Dropout(0.2))
# model.add(MaxPool1D(2))
          
# #model.add(Conv1D(256,5,activation='relu'))    
# model.add(GlobalMaxPool1D())
    
# model.add(Dense(256, activation='relu'))
# model.add(Dense(len(unique_y), activation='softmax'))
    
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# model.summary()

In [ ]:
# # Define the callback to save the best model during training:
# mc=ModelCheckpoint(mymodel, monitor='val_loss', mode='min', save_best_only=True,verbose=1)

# #Let’s train the model with a batch size of 128 for 50 epochs:
# history = model.fit(np.array(x_tr),np.array(y_tr),batch_size=128,epochs=50, validation_data=(np.array(x_val),np.array(y_val)),verbose=1, callbacks=[mc])

# # Load the best model:
# from keras.models import load_model
# model = load_model(mymodel)

# Start here if you don't want to improve the model

In [ ]:
# import random
# ind = np.random.randint(0,len(x_val)-1)

# # Load the model (if needed):
# if model == None:
#   from keras.models import load_model
#   model = load_model(mymodel)

# random_music = x_val[ind]

# predictions=[]
# for i in range(100):

#     random_music = random_music.reshape(1,no_of_timesteps)

#     prob  = model.predict(random_music)[0]
#     y_pred= np.argmax(prob,axis=0)
#     predictions.append(y_pred)

#     random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
#     random_music = random_music[1:]
    
# print(predictions)
# # view raw

In [ ]:
# # Now, we will convert the integers back into the notes.

# x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_x)) 
# predicted_notes = [x_int_to_note[i] for i in predictions]

# # The final step is to convert back the predictions into a MIDI file. 
# # Let’s define the function to accomplish the task.

In [ ]:
# def convert_to_midi(prediction_output):
   
#     offset = 0
#     output_notes = []

#     # create note and chord objects based on the values generated by the model
#     for pattern in prediction_output:
        
#         # pattern is a chord
#         if ('.' in pattern) or pattern.isdigit():
#             notes_in_chord = pattern.split('.')
#             notes = []
#             for current_note in notes_in_chord:
                
#                 cn=int(current_note)
#                 new_note = note.Note(cn)
#                 new_note.storedInstrument = instrument.Piano()
#                 notes.append(new_note)
                
#             new_chord = chord.Chord(notes)
#             new_chord.offset = offset
#             output_notes.append(new_chord)
            
#         # pattern is a note
#         else:
            
#             new_note = note.Note(pattern)
#             new_note.offset = offset
#             new_note.storedInstrument = instrument.Piano()
#             output_notes.append(new_note)

#         # increase offset each iteration so that notes do not stack
#         offset += 1
#     midi_stream = stream.Stream(output_notes)
#     midi_stream.write('midi', fp='music.mid')

In [ ]:
# # Convert the predictions into a musical file:
# convert_to_midi(predicted_notes)

# Grab the MIDI file and drop it into a Garageband track

In [ ]:
# !cp music.mid drive/MyDrive/AI/AIbach.mid ;